In [1]:
import numpy as np
import pandas as pd
import tqdm 

In [2]:
# TODO : 원하는 데이터 경로 지정하기
raw_data = pd.read_csv("/opt/ml/input/data/train_data.csv")
user_id = raw_data["userID"].unique() # 유저 아이디 목록 리스트로 저장

In [3]:
acc_by_kt_dict = dict()
"""
acc_by_kt_dict : 각 유저의 KT별 정답률을 저장한 dictionary
    - Key : 유저 아이딩
    - Value : 리스트 두개
        - 첫번째 리스트 : KT를 담고 있는 리스트
        - 두번째 리스트 : 첫번째 리스트에 동일한 인덱스에 있는 KT의 정답률
"""
for user in user_id :
    acc_by_kt_dict[user] = [[] for _ in range(2)] 

for user in tqdm.tqdm(user_id) : # 모든 유저에 대해
    data = raw_data[raw_data["userID"]==user] # 각 유저에 대한 행만 추출하여 data에 저장
    kt_l = data["KnowledgeTag"].unique() # 행들에 있는 고유한 KT 추출 ex) [7724, 7720, 7640]

    for i in kt_l : # 유저가 풀어본 각 KT 대해 
        kt = int(i) 
        kt_correct = 0
        kt_total = 0
        for j in np.where(data["KnowledgeTag"]==i)[0] : # 해당 KT인 문제만 순회하면서 
        #for j in range(len(data)) :
            if data["KnowledgeTag"].iloc[j] == i : # 동일한 KT에 대해 
                if data["answerCode"].iloc[j] == 1 : # 정답을 맞췄으면, correct, total count + 1
                    kt_correct+=1
                    kt_total+=1
                else : 
                    kt_total+=1 # 맞추지 못했으면 total count만 + 1
        acc_by_kt_dict[user][0].append(kt) # 해당 KT는 KT 리스트에 저장
        acc_by_kt_dict[user][1].append(kt_correct/kt_total) # KT에 대한 정답률 저장

100%|██████████| 6698/6698 [01:52<00:00, 59.55it/s] 


In [6]:
"""
acc_by_kt_category_list 
: 해당 리스트는 feature를 추가하기 전에 feature 값들을 일렬로 저장하기 위해 만든 리스트입니다.
"""
acc_by_kt_category_list = list() 

for i in tqdm.tqdm(range(len(raw_data))) : # 모든 행(instance)에 대해
    u = raw_data["userID"].iloc[i] # 유저를 확인하고
    kt = raw_data["KnowledgeTag"].iloc[i] # 대분류를 확인하고
    idx = np.where(acc_by_kt_dict[u][0]==kt)[0][0] # 해당 유저의 해당 대분류에 대한 정답률을 위에서 정의한 dictionary에서 찾아서
    acc = acc_by_kt_dict[u][1][idx]
    acc_by_kt_category_list.append(acc) # 리스트에 저장한다.

raw_data["KnowledgeTagAcc"] = acc_by_kt_category_list # 리스트에 KnowledgeTagAcc feature 추가

100%|██████████| 2266586/2266586 [01:22<00:00, 27577.00it/s]


In [7]:
raw_data

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,KnowledgeTagAcc
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1.0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1.0
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1.0
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1.0
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,1.0
...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,0.2
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1.0
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,1.0
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,1.0


In [9]:
"""
acc_cate_list : 각 대분류 정답률을 카테고리로 만들어 저장하려고 한 리스트. 
ex. 원래 값 [0.04, 0.79, 0.43, 0.99] -> 곱하기 10 : [0.4, 7.9, 4.3, 9.9] -> round : [0, 8, 4, 10] -> 더하기 1 : [1, 9, 5, 11]
"""
acc_cate_list = list()
for i in tqdm.tqdm(range(len(raw_data))) :
    acc = raw_data["KnowledgeTagAcc"].iloc[i] * 10
    acc = round(acc) + 1  
    acc_cate_list.append(acc) 

raw_data["KTAccuracyCate"] = acc_cate_list # 리스트로 저장

100%|██████████| 2266586/2266586 [00:28<00:00, 79239.93it/s]


In [10]:
raw_data

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,KnowledgeTagAcc,KTAccuracyCate
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1.0,11
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1.0,11
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1.0,11
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1.0,11
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,1.0,11
...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,0.2,3
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1.0,11
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,1.0,11
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,1.0,11


In [11]:
# TODO : 이름과 경로 원하는 데로 변경하기
raw_data.to_csv("train_data_with_KTAccuracy.csv", index=False) # 최종 결과 csv로 반환